In [ ]:
from src import main
ms = main.do_train()

import torch
from src import MODELS
runs_inx = 1
while (MODELS / f"runs_{runs_inx}.pth").is_file():
    runs_inx += 1
runs_savename = MODELS / f"runs_{runs_inx}.pth"
torch.save([x.models for x in ms], runs_savename)

In [7]:
from src import datasets
import torch
from collections import Counter
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rcParams["figure.dpi"] = 250
dm = datasets.MNISTDataModule()
ds = dm.get_test_dataset()
with torch.no_grad():
    signature_dists = None
    n_samples = 3
    while signature_dists is None or any(len(x) == 1 for x in signature_dists):
        signature_dists = []
        n_samples += 1
        sampling_inxs = list(range(n_samples))
        batch = torch.stack([ds[i][0] for i in sampling_inxs]).to("cuda")
        for model_ensemble in ms:
            ens_signature = []
            for model_out in model_ensemble.to("cuda")(batch).argmax(dim=-1):
                signature = tuple(model_out.tolist())
                ens_signature.append(signature)
            signature_dists.append(Counter(ens_signature))

labeled_signdists = {chr(ord("A")+i): dict(sd) for i, sd in enumerate(signature_dists)}
plot_order = list(labeled_signdists.keys())
n_ensembled = ms[0].n_ensembled
point_pairs = {}
for inx_a, labela in enumerate(plot_order):
    sda = labeled_signdists[labela]
    for labelb in plot_order[inx_a+1:]:
        sdb = labeled_signdists[labelb]
        pair_points = []
        for k, v in sda.items():
            pair_points.append((v / n_ensembled, sdb.get(k, 0) / n_ensembled))
        point_pairs[f"{labela}*{labelb}"] = pair_points

fig = plt.gcf()
ax = plt.gca()

for label, pts in point_pairs.items():
    ax.scatter(*zip(*pts), label=label)
ax.plot([0, 1], [0, 1])
fig.tight_layout()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_title(f"Inductive bias - {n_samples=}")
plt.savefig("output.png")
plt.legend()
plt.show()
print("Done plotting")